In [20]:
#import necessary modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [21]:
##SOURCE: https://dataaspirant.com/lasso-regression/#t-1606404715785

#import dataset for modeling
df = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/All scripts - paper/data/GlobalModelData/PredictingDataset.csv', sep=',')
df

,Unnamed: 0,Longitude,Latitude,mean_value_NO2,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,NDVI,trafBuf25,trafBuf50
0,0,10.008293,48.397079,29.250500,22.585377,13.402575,8123.319336,54316.64453,0.000000,203502.42190,4167.064941,0.000058,1.000663,2310.0,0.000000,0.000000
1,1,10.021227,53.523262,32.098936,23.546194,22.862799,9558.207031,76786.28125,0.000000,99596.04688,1850.278076,0.000081,0.978245,2518.0,0.000000,0.000000
2,2,10.053859,53.592301,58.703744,18.290140,16.493486,10471.541020,88439.60938,46.359188,265563.71880,2016.838379,0.000076,0.914638,3216.0,674.791667,674.791667
3,3,10.081685,53.529020,24.249276,23.645823,18.248226,11165.945310,76961.00000,0.000000,97921.56250,478.363281,0.000074,0.712983,1052.0,0.000000,0.000000
4,4,10.096292,48.847917,19.912116,14.026642,5.514785,6017.511230,30814.58984,0.000000,176265.68750,3338.221191,0.000047,1.000641,3240.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,477,9.979031,48.396836,27.221808,29.130899,12.865678,10829.854490,54732.38281,0.000000,198280.70310,3127.346436,0.000058,0.964609,2607.0,0.000000,0.000000
478,478,9.981599,53.529158,32.763475,33.719387,36.813637,9855.304688,83610.52344,0.000000,117941.50000,1480.759766,0.000088,1.000696,NaN,0.000000,0.000000
479,479,9.990569,53.507930,24.966811,17.919800,25.516184,9541.226562,76877.67188,0.000000,94117.64063,2670.007568,0.000082,0.485096,4338.0,0.000000,0.000000
480,480,9.997256,53.681993,39.256824,13.382681,7.228415,7302.048828,44593.98438,54.294807,171093.43750,2462.917969,0.000063,1.000694,4006.0,674.791667,674.791667


In [22]:
#remove unique idenifier & geodata
df = df.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)
#replace NaN's with 0's
df = df.fillna(0)

y = df['mean_value_NO2'] #specify target
x = df.drop(['mean_value_NO2'], axis=1) #predictors
feature_list = list(x.columns)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=20)
#random state is needed to ensure that same results are generated each time.

## ENSURING THAT X- AND Y-TRAINING SET CONTAIN SAME AMOUNT OF ROWS. SAME FOR TESTING.
print('Training Features Shape:',X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (361, 12)
Training Labels Shape: (361,)
Testing Features Shape: (121, 12)
Testing Labels Shape: (121,)


In [23]:
#determining alpha parameter
o = []
def alp(g):
    model_lasso = Lasso(alpha=g)
    model_lasso.fit(X_train, Y_train)
    pred_train_lasso= model_lasso.predict(X_train)
    pred_test_lasso= model_lasso.predict(X_test)
    
    ##RMSE
    print(g)
    def rmse(predictions, targets):
        return np.sqrt(((predictions - targets) ** 2).mean())
    
    print("testing rmse: ", rmse(pred_test_lasso, Y_test))

    ##R2
    print("R2 score testing data: ", r2_score(Y_test, pred_test_lasso))

    ##MEAN ABSOLUTE ERROR 
    print("MAE testing data: ", mean_absolute_error(Y_test, pred_test_lasso))

In [24]:
#examine model performance for each alpha parameter
o = np.arange(0.1, 1.1, 0.1)
store = []
for i in o:
    alp(i)

0.1
testing rmse:  8.679330270745401
R2 score testing data:  0.5883213701632493
MAE testing data:  6.3441714703317675
0.2
testing rmse:  8.707142403777999
R2 score testing data:  0.5856787686799584
MAE testing data:  6.369535081670903
0.30000000000000004
testing rmse:  8.707056814594516
R2 score testing data:  0.585686914002489
MAE testing data:  6.370681801372749
0.4
testing rmse:  8.706986711278377
R2 score testing data:  0.5856935855115746
MAE testing data:  6.371828521157294
0.5
testing rmse:  8.70693207455725
R2 score testing data:  0.5856987850768797
MAE testing data:  6.372975239257889
0.6
testing rmse:  8.706892940451182
R2 score testing data:  0.5857025092982504
MAE testing data:  6.374121958846036
0.7000000000000001
testing rmse:  8.706869279933523
R2 score testing data:  0.5857047609579051
MAE testing data:  6.375268677318315
0.8
testing rmse:  8.706861143105874
R2 score testing data:  0.5857055352999269
MAE testing data:  6.376415399039728
0.9
testing rmse:  8.7068684887334

In [25]:
## Build the lasso model with best alpha
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, Y_train)
pred_train_lasso= model_lasso.predict(X_train)
pred_test_lasso= model_lasso.predict(X_test)

In [26]:
##RMSE 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(pred_train_lasso, Y_train))
print("testing rmse: ", rmse(pred_test_lasso, Y_test))

##R2
print("R2 score training data: ", r2_score(Y_train, pred_train_lasso))
print("R2 score testing data: ", r2_score(Y_test, pred_test_lasso))

##MEAN ABSOLUTE ERROR 
print("MAE training data: ", mean_absolute_error(Y_train, pred_train_lasso))
print("MAE testing data: ", mean_absolute_error(Y_test, pred_test_lasso))

training rmse:  6.63857157662163
testing rmse:  8.679330270745401
R2 score training data:  0.7219958724562463
R2 score testing data:  0.5883213701632493
MAE training data:  4.909790180184455
MAE testing data:  6.3441714703317675
